<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/it_whisper_medium_test_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-2m4kbbjl
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-2m4kbbjl
  Resolved https://github.com/huggingface/transformers to commit 58e7f9bb2faf30622c9bead7adf472ac59f3d301
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8178366 sha256=a03eb869ab1bd9e3b64f88979befac1e62b216060eebae84edb73c0d1aed92e4
  Stored in directory: /tmp/pip-ephem-wheel-cache-bxcs3bs3/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [86]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [87]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [88]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "it", split="test")
#column_name = "audio, normalized_text"

In [89]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 1177
})

In [90]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [91]:
model = whisper.load_model("medium")

In [92]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [93]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [94]:
text

,0
0,questo è fondamentale se pensiamo a paesi come...
1,allora punto numero uno inserire l'impegno del...
2,qui naturalmente c'è chi ha la verità in tasca...
3,il g otto e il g venti delle settimane scorse ...
4,signora presidente signor commissario potonik ...
...,...
1172,signor presidente soltanto un'informazione per...
1173,per fermare queste contrapposizioni che si svo...
1174,siamo quasi in un regime di dittatura europea ...
1175,signor presidente onorevoli colleghi nella rel...


In [95]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [96]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,questo è fondamentale se pensiamo a paesi come...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0203857421875, -0.037109375, 0.000793457031...",16000
1,female,allora punto numero uno inserire l'impegno del...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.013397216796875, 0.02423095703125, 0.036041...",16000
2,male,qui naturalmente c'è chi ha la verità in tasca...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0390625, 0.035186767578125, 0.0305786132812...",16000
3,male,il g otto e il g venti delle settimane scorse ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, 9.1552734375e-05, -6.10351...",16000
4,male,signora presidente signor commissario potonik ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000640869140625, 0.000457763671875, -0.0014...",16000
...,...,...,...,...,...
1172,male,signor presidente soltanto un'informazione per...,/root/.cache/huggingface/datasets/downloads/ex...,"[-9.1552734375e-05, -0.000518798828125, -0.005...",16000
1173,male,per fermare queste contrapposizioni che si svo...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00311279296875, 0.003936767578125, 0.003112...",16000
1174,male,siamo quasi in un regime di dittatura europea ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.02105712890625, 0.024627685546875, 0.033691...",16000
1175,male,signor presidente onorevoli colleghi nella rel...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.004058837890625, 0.004150390625, 0.00357055...",16000


In [97]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,questo è fondamentale se pensiamo a paesi come...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0203857421875, -0.037109375, 0.000793457031...",16000
2,male,qui naturalmente c'è chi ha la verità in tasca...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0390625, 0.035186767578125, 0.0305786132812...",16000
3,male,il g otto e il g venti delle settimane scorse ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, 9.1552734375e-05, -6.10351...",16000
4,male,signora presidente signor commissario potonik ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000640869140625, 0.000457763671875, -0.0014...",16000
5,male,la restituzione dei beni rubati deve essere in...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001953125, -0.003204345703125, 0.0010375976...",16000
...,...,...,...,...,...
1172,male,signor presidente soltanto un'informazione per...,/root/.cache/huggingface/datasets/downloads/ex...,"[-9.1552734375e-05, -0.000518798828125, -0.005...",16000
1173,male,per fermare queste contrapposizioni che si svo...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00311279296875, 0.003936767578125, 0.003112...",16000
1174,male,siamo quasi in un regime di dittatura europea ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.02105712890625, 0.024627685546875, 0.033691...",16000
1175,male,signor presidente onorevoli colleghi nella rel...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.004058837890625, 0.004150390625, 0.00357055...",16000


In [98]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
1,female,allora punto numero uno inserire l'impegno del...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.013397216796875, 0.02423095703125, 0.036041...",16000
11,female,quindi problemi sul piano della sicurezza sul ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001220703125, -0.001068115234375, -0.00131...",16000
12,female,questo è vero per i cristiani in medio oriente...,/root/.cache/huggingface/datasets/downloads/ex...,"[3.0517578125e-05, -0.000640869140625, -0.0004...",16000
14,female,di una guerra per procura sulla libia e in lib...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000213623046875, -0.000823974609375, -0.00...",16000
18,female,ricordo che tra gli obiettivi più ampi delle n...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.003387451171875, -0.001373291015625, -0.00...",16000
...,...,...,...,...,...
1145,female,scarse informazioni per i passeggeri assistenz...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00341796875, -0.001953125, -0.002227783203...",16000
1149,female,signora presidente onorevoli colleghi ringrazi...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000823974609375, -0.000457763671875, -0.000...",16000
1152,female,la libia ha grandissime capacità quello che l'...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0462646484375, 0.049285888671875, 0.0446777...",16000
1165,female,la necessità di proteggere tutte le comunità e...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.10955810546875, -0.15838623046875, -0.2659...",16000


In [99]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="it")
    return result["text"]

In [100]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [101]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 1 files, WER: 1.1515151515151516


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 2 files, WER: 0.25806451612903225


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

Processed 873 files, WER: 1.1764705882352942


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 874 files, WER: 0.5625


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 875 files, WER: 0.24528301886792453
Average WER: 0.3509437545367378


In [102]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 1 files, WER: 0.32432432432432434


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 2 files, WER: 0.3958333333333333


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

Processed 300 files, WER: 0.35135135135135137


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 301 files, WER: 0.06666666666666667


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Processed 302 files, WER: 0.22916666666666666
Average WER: 0.28446579592015014
